In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import math as m
import pandas as pd

In [355]:
# load of our Iris data 
iris= datasets.load_iris()
print(iris)
df = pd.DataFrame(iris.data)  # Giving it the data Frame
Y = pd.DataFrame(iris.target)  #  our output of the iris data  
df.columns = ["sl", "sw", 'pl', 'pw']
features   = ["sl", "sw", 'pl', 'pw']
# splitting of data features into x_train y_train x_test y_test 
x_train, x_test, y_train, y_test = train_test_split(df,Y,random_state = 1)
x_train=x_train.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)
x_test=x_test.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)



In [356]:
# node is  made to make the tree for the given data 
class node:
    def __init__(self,f=None,fv=None,gr=-1,en=-1,val=-1):
        self.entropy=en      # Present Entropy
        self.f_to_split=f    # Feature about which the current node will be splitted (if not a leaf node)
        self.feature_val=fv  # The label of the feature about which the data was split to get the present node 
        self.gain_ratio=gr   # Gain Ratio by which it is gonna be splitted
        self.children = []   # Initialising children as an empty list
        self.midF = val      # data to store mid value  
        self.array = []      # array to store zeros ones and twos classes of the output

In [357]:
# It is function for the partition of the continues data so that we can the best possible mid value so that we can spilt our data in  to two best features   
def Partition(array,midPoint,Y,k):
    # ans1 store the data for the values less tha mid 
    # ans2 store the values for the data more than or equal to mid
    ans1= np.zeros(3,dtype=int)
    ans2= np.zeros(3,dtype=int)
    Y = np.array(Y)
    # loop over the array(with bestfeature) and Y is the output for our bestFeature 
    for i in range(array.shape[0]):
        if array[i] < midPoint:
            if Y[i] == 0:
                ans1[0]+=1
            elif Y[i] == 1:
                ans1[1]+=1
            elif Y[i]==2:
                ans1[2]+=1
        else:
            if Y[i] == 0:
                ans2[0]+=1
            elif Y[i] == 1:
                ans2[1]+=1
            elif Y[i]==2:
                ans2[2]+=1
    return ans1,ans2            
    

In [358]:
def info_Gain(ans):
    value = ans[0] + ans[1] +ans[2]
    info=0
    for i in range(3):
        if value == 0:
            X=0
        if value!=0:
            X = ans[i]/value
        if X == 0:
            info += 0
        if X!=0:
            info += (-1)*(X * m.log(X))
    return info,value    

In [359]:
# This the function which will give us the InfoGain or Entropy  
def Info_Gain(X,Y,selFtre):
    # array with bestFeature 
    array = X[selFtre]
    maxGain = -1   # maxgain is the data which will us the best infoGain or entropy   
    mid = -1       
    for i in range(array.shape[0]-1):
        
        #################################################################################################################
        
        # midPoint cantain the mid value to consecuitive data points  
        midPoint = (array[i] + array[i+1] ) / 2
        # ans1  and ans2 contain the output divided on the bases of our midpoint
        ans1,ans2 = Partition(array,midPoint,Y,i)
        # ans3  cantain classes  ouput on the bases of which ans1 and ans2 is created 
        ans3 = ans1+ans2

        ###################################################################################################################
        #info_gain is the formula applied on basis of which entropy is given to us  
        # value1 value2 value3  contain the number on which our data is divided
        InfoGain1,value1 = info_Gain(ans1)
        InfoGain2,value2 = info_Gain(ans2)
        InfoGain3,value3 = info_Gain(ans3)
        ###################################################################################################################
        # maxG and value is the formula applied to caluclate the macGain
        value = ((value1/value3)*InfoGain1) + ((value2/value3)*InfoGain2)
        maxG = InfoGain3 - value
        ###################################################################################################################
        
        # To get the maxGain due to continuous values in features 
        if maxGain <= maxG:
            maxGain = maxG
            mid = midPoint
            valueArray = np.array([value1,value2,value3])   
        
        
    return maxGain , valueArray , mid 
        
    

In [360]:
# This is the formula to calculate the splitInfo to calulate the gain ratio
def Split_Info(X,Y,valueArray):
    x = valueArray[0] / valueArray[2]
    y = valueArray[1] / valueArray[2]
    ans = ((-x) * m.log(x)) + ((-y) * m.log(y))
    return ans 

In [361]:
# this the Fuction where we calculate the gain ratio 
def gain(X,Y,selFtre):
    infoGain,valueArray, mid  = Info_Gain(X,Y,selFtre)
    splitInfo = Split_Info(X,Y,valueArray)
    Gain = infoGain /splitInfo
    return Gain,mid,infoGain

In [1]:
# This the main fiunction where we get our decision tree
def decision_tree(X,Y,features,level,Entropy,root,mid):
    # This is our Base case where we reach to our base or leaf node 
    m = set(Y[0])
    if len(m)==1 and (list(m)[0] == 1 or list(m)[0] == 2  or list(m)[0] == 0):  # base case if node becomes pure 

        # This is the place where we store our data into the node 
        root.midF  =  mid
        root.array.append(len(Y[Y[0]==0]))
        root.array.append(len(Y[Y[0]==1]))
        root.array.append(len(Y[Y[0]==2]))
        root.entropy    = Entropy

        # This the place where we print our node level wise 
        print("Level = " + str(level))
        print("Count of  0 = " + str(len(Y[Y[0]==0])))
        print("Count of  1 = " + str(len(Y[Y[0]==1])))
        print("Count of  2 = " + str(len(Y[Y[0]==2])))
        print("Current Entropy = " + str(Entropy))
        print("Reached the LEAF NODE")
        print()
        
        return
    
    if len(features) == 0: 
        # This is the place where we store our data into the node 
        
        root.midF  =  mid
        root.array.append(len(Y[Y[0]==0]))
        root.array.append(len(Y[Y[0]==1]))
        root.array.append(len(Y[Y[0]==2]))
        
        
         # This the place where we print our node level wise 
        print("Level = " + str(level))
        print("Count of  0 = " + str(len(Y[Y[0]==0])))
        print("Count of  1 = " + str(len(Y[Y[0]==1])))
        print("Count of  2 = " + str(len(Y[Y[0]==2])))
        print("Current Entropy = " + str(Entropy))
        print("Reached the LEAF NODE")
        print()
        
        return
    
    # place were we define our variable 
    
    maxGain = 0  # Is the variable which will get us the best gain ratio of the particular feature
    mid = 0      # It is varaible which will give us the value on which we to split the our feature into two part
    bestFeature = ""   # variable to get us the betsFeature   
    Entropy=0      # variable to get us the entropy 
    
    # loop on our feature list to get our bestfeature 
    
    for f in features:
        # ans contain all the value Gainratio midvalue and the Entropy for the particular feature 
        ans = gain(X,Y,f)
        # we compare the Maxgain to get ou best gain ratio midvalue and entropy
        
        if maxGain <= ans[0]:
            maxGain = ans[0]
            mid = ans[1]
            bestFeature = f
            Entropy = ans[2]
    
    # Place we store our data into the node  
    root.entropy    = Entropy
    root.f_to_split = bestFeature
    root.midF       = mid
    root.children.append(node(fv=bestFeature))
    root.array.append(len(Y[Y[0]==0]))
    root.array.append(len(Y[Y[0]==1]))
    root.array.append(len(Y[Y[0]==2]))
                    
    # Place where we print our data  
    print("Level = " + str(level))
    print("Count of  0 = " + str(len(Y[Y[0]==0])))
    print("Count of  1 = " + str(len(Y[Y[0]==1])))
    print("Count of  2 = " + str(len(Y[Y[0]==2])))
    print("Current Entropy is = " + str(Entropy))
    print("Splitting on feature " + str(bestFeature)  + " with gain ratio = " + str(maxGain))
    print()
    
    # PLace where we split our data on the basis of bestFeature so that we can pass our data to the next level 
    X1 = X[X[bestFeature]<mid]
    Y1 = Y.iloc[X1.index]
    X1 = X1.reset_index(drop=True)
    Y1 = Y1.reset_index(drop=True)
    X1 = X1.drop([bestFeature],axis=1)

    # PLace where we split our data on the basis of bestFeature so that we can pass our data to the next level 
    X2 = X[X[bestFeature]>=mid]
    Y2 = Y.iloc[X2.index]
    X2 = X2.reset_index(drop=True)
    Y2 = Y2.reset_index(drop=True)
    X2 = X2.drop([bestFeature],axis=1)

    features.remove(bestFeature) # Here we remove the feature which we have used 
    
    decision_tree(X1,Y1,features,level+1,Entropy,root.children[0],mid) # We call our feature to get us the split on our next featsure  
    
    root.children.append(node(fv=bestFeature)) # here we append new node in our children so that we call save new data for parent node 
    
    decision_tree(X2,Y2,features,level+1,Entropy,root.children[1],mid) # We call our feature to get us the split on our next featsure  
    
    return 
     


In [363]:
# here we get the our root node or our first node 
root = node()

In [364]:
# this the function to predict our output fo X_test input 

def predict(X,root,Y,decide):
    
    # this the base case were we reach to leaf node and decide our output 
    if len(root.children) == 0:

        bstFtre = root.feature_val
        mid = root.midF
        
        # if our decide is equal to zero then we consider pur left node 
        if decide == 0:
            count1 = max(root.array)  # <mid  count1 contain cvalue with maximum in  class 
            X1 = X[X[bstFtre]<mid]
            for j in range(3):
                if count1 == root.array[j]:
                    Y.iloc[X1.index] = j   
        
        # other wise if our decoide is equla to one then we consider right node              
        else:
            count2 = max(root.array)  #>mid count2 contain cvalue with maximum in  class 
            X2 = X[X[bstFtre]>=mid]
            for j in range(3):
                if count2 == root.array[j]:
                    Y.iloc[X2.index] = j
                    
        return Y
    
    
    mid = root.midF
    bstFtre = root.f_to_split
    X1 = X[X[bstFtre]<mid]
    # here we pass our next data for the next level were data value is less than mid 
    predict(X1,root.children[0],Y,0)
    X2 = X[X[bstFtre]>=mid]
    # here we pass our next data for the next level were data value is more than mid 
    predict(X2,root.children[1],Y,1)
    
    
    return Y

In [365]:
# this the fuction which compare our predicted data and actual data how accurate is our prediction
def accuracy(root,x_test,y_test,Y,val):
    y_pred = predict(x_test,root,Y,val)
    return np.array(y_pred==y_test).sum()/len(y_pred)

In [366]:
# this the fuction where we call our decision tree 
decision_tree(x_train,y_train,features,0,0,root,0)

Level = 0
Count of  0 = 37
Count of  1 = 34
Count of  2 = 41
Current Entropy is = 0.6344312705534911
Splitting on feature pw with gain ratio = 1.0

Level = 1
Count of  0 = 37
Count of  1 = 0
Count of  2 = 0
Current Entropy = 0.6344312705534911
Reached the LEAF NODE

Level = 1
Count of  0 = 0
Count of  1 = 34
Count of  2 = 41
Current Entropy is = 0.4210281233360511
Splitting on feature pl with gain ratio = 0.6255881494700155

Level = 2
Count of  0 = 0
Count of  1 = 29
Count of  2 = 1
Current Entropy is = 0.09993493397123412
Splitting on feature sl with gain ratio = 0.40801422054726605

Level = 3
Count of  0 = 0
Count of  1 = 1
Count of  2 = 1
Current Entropy is = 0.6931471805599453
Splitting on feature sw with gain ratio = 1.0

Level = 4
Count of  0 = 0
Count of  1 = 1
Count of  2 = 0
Current Entropy = 0.6931471805599453
Reached the LEAF NODE

Level = 4
Count of  0 = 0
Count of  1 = 0
Count of  2 = 1
Current Entropy = 0.6931471805599453
Reached the LEAF NODE

Level = 3
Count of  0 = 0
C

In [367]:
# Y is the array which contain all zeros we pass it so that predicted value can be saved in it 
Y = np.zeros(y_test.shape[0],dtype=int)
Y = pd.DataFrame(Y)
# this print us the accurracy how accurrate is our formula 
print(accuracy(root,x_test,y_test,Y,-1))

0.9736842105263158
